In [2]:
def compute_features(channel_name):
    path = 'G:/EEG Project/Hut lab data/30_sec_epochs/data_raw/'
    channel= channel_name
    start = time.time()
    print('\n channel: ', channel)
    csv_files = glob.glob(path + '*'+channel+'.csv')
    score_files = glob.glob(path + '*txt.csv')
    
    cwt_spectral(csv_files, score_files, channel)
    AR_coeff(csv_files, score_files, channel)
    temporal(csv_files, score_files, channel)
    
    end = time.time()
    elapsed = (end - start)/60
    print("Time to complete: %.2f" % elapsed, "mins")
    
    
def cwt_spectral(csv_files, score_files, channel):
    for csv, txt in zip(csv_files, score_files):
        name_feature = 'cwt_spectral'
        cohl_feature = []
        df = pd.read_csv(csv, index_col=0)
        df.fillna(0, inplace=True)
        tf = pd.read_csv(txt, index_col=0)['Score']
        size = int(df.size/len(df))

        for i in range(size):
            np.seterr(all='ignore')
            epoch = np.asarray(df.iloc[:, i])
            epoch = epoch[~np.isnan(epoch)]

            fs = 128
            dt = 1/fs
            wave, scales, freqs, coi, fft, fftfreqs = pycwt.wavelet.cwt(epoch, dt)   #, dj, s0, J, mother
            cohl = np.abs(wave)
            cohl = cohl[::-1]
            cohl_flat= cohl.flatten()
            splits = [318720, 360960, 395520, 445440]
            cwt_spectral = np.split(cohl_flat, splits)

            cohl_m = []
            cohl_mcr = []
            cohl_p = []
            cohl_c = []
            cohl_f = []
            cohl_v = []
            cohl_r = []

            for s in range(5):
                cohl_m.append(np.mean(cwt_spectral[s]))
                cohl_mcr.append((np.nonzero(np.diff(cwt_spectral[s] > np.mean(cwt_spectral[s])))[0]).size)
                cohl_p.append(np.trapz(cwt_spectral[s]))
                cohl_c.append(np.sum(cwt_spectral[s]*len(cwt_spectral[s]))/np.sum(cwt_spectral[s]))
                cohl_f.append(scp.stats.mstats.gmean(cwt_spectral[s])/np.mean(cwt_spectral[s]))
                cohl_v.append(np.var(cwt_spectral[s]))
                cohl_r.append(0.95*np.sum(cwt_spectral[s]))

            cohl_feature.append([cohl_m[0], cohl_mcr[0], cohl_p[0], cohl_c[0], cohl_f[0], cohl_v[0], cohl_r[0],
                                cohl_m[1], cohl_mcr[1], cohl_p[1], cohl_c[1], cohl_f[1], cohl_v[1], cohl_r[1], 
                                cohl_m[2], cohl_mcr[2], cohl_p[2], cohl_c[2], cohl_f[2], cohl_v[2], cohl_r[2],
                                cohl_m[3], cohl_mcr[3], cohl_p[3], cohl_c[3], cohl_f[3], cohl_v[3], cohl_r[3], 
                                cohl_m[4], cohl_mcr[4], cohl_p[4], cohl_c[4], cohl_f[4], cohl_v[4], cohl_r[4]])
            sys.stdout.write("\r%d%% epochs processed" % i)

        new_df = pd.DataFrame(cohl_feature)                               
        new_df.columns = ['Dm', 'Dmcr', 'Dp', 'Dc', 'Df' ,'Dv', 'Dr', 
                         'Tm', 'Tmcr', 'Tp', 'Tc', 'Tf' ,'Tv', 'Tr',
                         'Am', 'Amcr', 'Ap', 'Ac', 'Af' ,'Av', 'Ar', 
                         'Bm', 'Bmcr', 'Bp', 'Bc', 'Bf' ,'Bv', 'Br',
                         'Gm', 'Gmcr', 'Gp', 'Gc', 'Gf' ,'Gv', 'Gr']

        big_df = pd.concat([new_df, tf], axis=1)
        big_df = big_df[~np.isnan(big_df['Score'])]
        big_df = big_df[big_df.Score < 7]
        big_df = big_df[~np.isnan(big_df['Gr'])]
        toPrint = csv.split('raw\\', 1)[1]
        toPrint = toPrint.split('.edf', 1)[0]
        big_df.to_csv('G:/EEG Project/Hut lab data/30_sec_epochs/cwt_spectral features/'+channel+'/'+toPrint+'_'+name_feature+'.csv')

        
def AR_coeff(csv_files, score_files, channel):
    for csv, txt in zip(csv_files, score_files):
        name_feature = 'AR_coeff'
        AR_coeff = []
        df = pd.read_csv(csv, index_col=0)
        df.fillna(0, inplace=True)
        tf = pd.read_csv(txt, index_col=0)['Score']
        size = int(df.size/len(df))

        for i in range(size):
            np.seterr(all='ignore')
            epoch = np.asarray(df.iloc[:, i])
            epoch = epoch[~np.isnan(epoch)]
            order_AR = 7
            AR, var, k = aryule(epoch, order_AR)
            AR_coeff.append(AR)            
            
            
        new_df = pd.DataFrame(AR_coeff)                               
        new_df.columns = ['AR_1','AR_2','AR_3','AR_4','AR_5','AR_6','AR_7']
        big_df = pd.concat([new_df, tf], axis=1)
        big_df = big_df[~np.isnan(big_df['Score'])]
        big_df = big_df[big_df.Score < 7]
        big_df = big_df[~np.isnan(big_df['AR_7'])]
        print('\n Writing feature set to .csv file...')
        toPrint = csv.split('raw\\', 1)[1]
        toPrint = toPrint.split('.edf', 1)[0]
        big_df.to_csv('G:/EEG Project/Hut lab data/30_sec_epochs/AR features/'+channel+'/'+toPrint+'_'+name_feature+'.csv')
   
def temporal(csv_files, score_files, channel):
    for csv, txt in zip(csv_files, score_files):
        name_feature = 't_features'
        t_features = []
        df = pd.read_csv(csv, index_col=0)
        df.fillna(0, inplace=True)
        tf = pd.read_csv(txt, index_col=0)['Score']
        size = int(df.size/len(df))

        for i in range(size):
            np.seterr(all='ignore')
            epoch = np.asarray(df.iloc[:, i])
            epoch = epoch[~np.isnan(epoch)]
            
            mean = np.mean(epoch)
            std = np.std(epoch)
            minimum = np.amin(epoch)
            maximum = np.amax(epoch)
            ZCR = (np.nonzero(np.diff(epoch > 0))[0]).size
            first_deriv = np.diff(epoch)
            second_deriv = np.diff(epoch,2)

            var_zero = np.mean(epoch ** 2)
            var_d1 = np.mean(first_deriv ** 2)
            var_d2 = np.mean(second_deriv ** 2)

            activity = var_zero
            morbidity = np.sqrt(var_d1 / var_zero)
            complexity = np.sqrt(var_d2 / var_d1) / morbidity        
            t_features.append([mean, std, minimum, maximum, ZCR, morbidity, complexity])
            
        new_df = pd.DataFrame(t_features)                               
        new_df.columns = ['mean', 'std', 'min', 'max', 'ZCR', 'H_mobility', 'H_complexity']
        big_df = pd.concat([new_df, tf], axis=1)
        big_df = big_df[~np.isnan(big_df['Score'])]
        big_df = big_df[big_df.Score < 7]
        big_df = big_df[~np.isnan(big_df['ZCR'])]
        print('\n Writing feature set to .csv file...')
        toPrint = csv.split('raw\\', 1)[1]
        toPrint = toPrint.split('.edf', 1)[0]
        big_df.to_csv('G:/EEG Project/Hut lab data/30_sec_epochs/AR features/'+channel+'/'+toPrint+'_'+name_feature+'.csv')
   

def non_linear():
    '''
    define the non-linear functins here as and then we can run the whole damn thing as one piece of code parallely
    '''
    return None